# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
inputFile = "../WeatherPy/output_data/cities.csv"

weatherData_df = pd.read_csv(inputFile)
weatherData_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tarko-Sale,0,RU,1603760310,93,64.92,77.78,4.42,12.71
1,Tiksi,100,RU,1603760310,94,71.69,128.87,6.26,5.46
2,Kavieng,20,PG,1603760310,60,-2.57,150.80,85.64,5.28
3,Hobart,75,AU,1603760265,62,-42.88,147.33,55.99,11.41
4,Nikolskoye,90,RU,1603760310,87,59.70,30.79,50.00,6.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure GMaps
gmaps.configure(api_key = g_key)

#Declare variables for future use
locations = weatherData_df[["Lat", "Lng"]]
humidity = weatherData_df["Humidity"]

In [9]:
#plot heatmap
heatMap = gmaps.figure(center = (48.0, -5.0), zoom_level = 2)
maxIntesity = np.max(humidity)

#Create Heat Layer
heatLayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 3)

#Add Layer
heatMap.add_layer(heatLayer)

#Display the Heat map
heatMap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Clean up and narrow down the dataframe to have only valid inputs
#and to fit our criteria
cleanedCity_df = weatherData_df.loc[(weatherData_df["Wind Speed"] <= 5) & (weatherData_df["Cloudiness"] <= 5) & (weatherData_df["Max Temp"] > 70) & (weatherData_df["Max Temp"] < 85)].dropna()

cleanedCity_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,Karratha,0,AU,1603760312,35,-20.74,116.85,82.74,2.42
47,Maués,0,BR,1603760313,68,-3.38,-57.72,81.34,2.35
103,Salalah,0,OM,1603760317,60,17.02,54.09,75.20,4.50
166,Morondava,0,MG,1603760321,68,-20.28,44.28,74.25,2.04
176,Lianzhou,0,CN,1603760322,67,24.78,112.38,70.05,4.61
212,Bhatkal,0,IN,1603760325,77,13.97,74.57,76.03,3.60
237,Porbandar,0,IN,1603760326,42,21.64,69.61,80.06,4.09
306,Óbidos,0,BR,1603760331,62,-1.92,-55.52,83.55,2.48
368,Grand Gaube,0,MU,1603760073,77,-20.01,57.66,72.00,4.00
454,Téra,0,NE,1603760340,15,14.01,0.75,79.88,3.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
#Create new dataframe "hotel_df"
hotel_df = cleanedCity_df[["City", "Country", "Lat", "Lng"]]

#Add the "Hotel Name" Column
hotel_df["Hotel Name"] = ""

#Display the dataframe
hotel_df

<ipython-input-27-a7f46abd4ef2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
25,Karratha,AU,-20.74,116.85,
47,Maués,BR,-3.38,-57.72,
103,Salalah,OM,17.02,54.09,
166,Morondava,MG,-20.28,44.28,
176,Lianzhou,CN,24.78,112.38,
212,Bhatkal,IN,13.97,74.57,
237,Porbandar,IN,21.64,69.61,
306,Óbidos,BR,-1.92,-55.52,
368,Grand Gaube,MU,-20.01,57.66,
454,Téra,NE,14.01,0.75,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_)

# Display figure
